# Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the Data (don't run this part again)

In [ ]:
data1 = pd.read_excel('./hotel_revenue_historical_full.xlsx', sheet_name='2018')
data1.head()

In [ ]:
data2 = pd.read_excel('./hotel_revenue_historical_full.xlsx', sheet_name='2019')
data2.head()

In [ ]:
data3 = pd.read_excel('./hotel_revenue_historical_full.xlsx', sheet_name='2020')
data3.head()

In [ ]:
data4 = pd.read_excel('./hotel_revenue_historical_full.xlsx', sheet_name='meal_cost')
data4.head()

In [ ]:
data5 = pd.read_excel('./hotel_revenue_historical_full.xlsx', sheet_name='market_segment')
data5.head(8)

In [ ]:
data6 = pd.read_excel('./country.xlsx')
data6.head()

In [ ]:
data1.to_csv('HRH2018.csv', index=False)

In [ ]:
data2.to_csv('HRH2019.csv', index=False)

In [ ]:
data3.to_csv('HRH2020.csv', index=False)

In [ ]:
data4.to_csv('meal_cost.csv', index=False)

In [ ]:
data5.to_csv('market_segment.csv', index=False)

In [ ]:
data6.to_csv('country.csv', index=False)

In [ ]:
d1 = pd.read_csv('./data/HRH2018.csv')
#d1.info()

In [ ]:
d2 = pd.read_csv('./data/HRH2019.csv')
#d2.info()

In [ ]:
d3 = pd.read_csv('./data/HRH2020.csv')
#d3.info()

In [ ]:
d = [d1, d2, d3]
res = pd.concat(d)

In [ ]:
print(res.shape)

In [ ]:
res.to_csv('./data/hotel_revenue.csv', index=False)

# Data Processing

In [ ]:
data = pd.read_csv('./data/hotel_revenue.csv')
pd.set_option('display.max_column', None)
data.head()
# data.info()

## Check data types

In [ ]:
data.info()

Data type of agent and company should be object, since the values represent the ID of the agent that made the booking and the ID of the company respectively.

To change the data type from float to object/string, change them into integer first to avoid the decimal-form values. To do so, null values in those columns should be filled with a number, 0.

Data type of children also should be integer and the null values in children column means there is no children, we assigned 0 into it as well (8 of null values).

In [ ]:
# Convert columns to appropriate datatypes
data['company'] = data['company'].fillna(0).astype(int)
data['company'] = data['company'].astype(str)
data.loc[data['company'] == '0', "company"] = np.nan

data['agent'] = data['agent'].fillna(0).astype(int)
data['agent'] = data['agent'].astype(str)
data.loc[data['agent'] == '0', "agent"] = np.nan

data['children'] = data['children'].fillna(0).astype(int)

## Handle Missing Values

In [ ]:
data.isnull().sum()/len(data)*100

Since the missing values of company column is up to 94% --> can drop this column and drop some unrelated columns as well

In [ ]:
data.drop(['company', 'reserved_room_type','assigned_room_type', 'reservation_status_date'], axis=1, inplace=True)

The null values in agent column possibly means that the booking wasn't through any agent, then we assigned "No Agent" into it as a value.

In [ ]:
data['agent'] = data['agent'].fillna("No Agent")

Fill null values in country column with the highest frequency value (mode) (625 of blank values)

In [ ]:
data['country'] = data['country'].fillna(data['country'].mode()[0])

In [ ]:
data.isnull().sum()/len(data)*100

## Handle Invalid Values

In [ ]:
# get the categorical and numeric column names
num_cols = data.select_dtypes(include = 'number').columns
cat_cols = data.select_dtypes(include = 'object').columns

In [ ]:
# statictic descriptive categorical columns
data[cat_cols].describe().T

1. Replace 'Undefined' values in meal column with highest frequency value 'BB'

In [ ]:
data['meal'].value_counts()

In [ ]:
data.loc[data['meal']=='Undefined', 'meal'] = 'BB'

2. Replace 'Undefined' values in market_segment with the highest frequency value 'Online TA'

In [ ]:
data['market_segment'].value_counts()

In [ ]:
data.loc[data['market_segment']=='Undefined', 'market_segment'] = 'Online TA'

3. Replace 'Undefined' values in distribution_channel with 'TA/TO'

In [ ]:
data['distribution_channel'].value_counts()

In [ ]:
data.loc[data['distribution_channel'] == 'Undefined', 'distribution_channel'] = 'TA/TO'

4. Replace 'CN' values in country with 'CAN'

In [ ]:
data['country'].value_counts()

In [ ]:
data.loc[data['country'] == 'CN', 'country'] = 'CAN'

In [ ]:
# statictic descriptive numerical columns
data[num_cols].describe().T

5. Replace 53 in arrival_date_week_number to 52, because there are only 52 weeks in a year.

In [ ]:
data[data['arrival_date_week_number'] == 53].shape

In [ ]:
data.loc[data['arrival_date_week_number'] == 53.0, "arrival_date_week_number"] = 52

6. Delete row data consist of negative ADR (average daily rate), because it's not possible that room price is negative

In [ ]:
data[data['adr'] < 0].shape

In [ ]:
data.drop(data[data['adr'] < 0].index, inplace=True)

7. Delete row data with ADR 5,400, because it seems like a data entry error.

In [ ]:
data[data['adr'] == 5400.0].shape

In [ ]:
plt.figure(figsize=(4,4))
sns.boxplot(y=data['adr'], color='b', orient='v')
plt.tight_layout()
plt.show()

In [ ]:
data.drop(data[data['adr'] == 5400.0].index, inplace=True)

8. Delete row data with 0 in adults column, because most of hotels won't permit anybody younger than 18 to stay in a room unaccompanied.

In [ ]:
data[data['adults'] == 0].shape

In [ ]:
data.drop(data[data['adults'] == 0].index, inplace=True)

In [ ]:
data.info()

In [ ]:
data.to_csv('./data/hotel_revenue_edited.csv', index=False)

In [ ]:
df = pd.read_csv('./data/hotel_revenue_edited.csv')
df.shape